In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# Create 1st serial acquisition source:
acq_arduino = LDAQ.serial_communication.SerialAcquisition(port="COM3", baudrate=250000, 
                                                acquisition_name="arduino",
                                                byte_sequence=(("int16", 2), ),
                                                start_bytes=  b"\xfa\xfb",
                                                end_bytes= b"\n",
                                                sample_rate=500,
                                                channel_names=["channel 1", "channel 2"])

# Here the arduino sends out bytes over serial where a line would look like this:
# b"\xfa\xfb\x00\x01\x00\x02\n

# Explanation:
# b"\xfa\xfb" are the start bytes
# b"\n" is the end byte
# b"\x00\x01" is the first sample (int16)
# b"\x00\x02" is the second sample (int16)

In [4]:
# Create visualization:
vis = LDAQ.Visualization(refresh_rate=30)
vis.add_lines((0, 0), "arduino", [0]) # channel 1
vis.add_lines((1, 0), "arduino", [1]) # channel 2

vis.config_subplot((0, 0), t_span=0.5)
vis.config_subplot((1, 0), t_span=0.5)

In [8]:
# create core object and add acquisition sources:
ldaq = LDAQ.Core(acquisitions=[acq_arduino], visualization=vis)
# set trigger:
ldaq.set_trigger(
    source  = "arduino",
    channel = "channel 2",
    level   = 100,
    duration= 60.) # if check event not triggered, measurement will run for 60 seconds

# add a check event that will stop the measurement:
sensor_abs_sum = 0
def end_measurement(self):
    """Will stop the measurement more than 1000 samples have been read."""
    acq = self.acquisitions[self.acquisition_names.index("arduino")] # get acquisition object
    time, data = acq.get_data(data_to_return="data") # get data
    
    global sensor_abs_sum
    sensor_abs_sum += np.sum(np.abs(data[:, 0])) # add absolute values of first channel to global variable
    
    if sensor_abs_sum > 1e8:
        return True
    else:
        return False
     
ldaq.add_check_events(end_measurement)

# run acquisition:
ldaq.run()

In [9]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement

{'arduino': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 6.512e+00, 6.514e+00,
         6.516e+00]),
  'channel_names': ['channel 1', 'channel 2'],
  'data': array([[ 512.,  576.],
         [ 512.,  639.],
         [ 512.,  700.],
         ...,
         [-512.,  906.],
         [-512.,  944.],
         [-512.,  975.]]),
  'sample_rate': 500}}